In [1]:
import os

In [2]:
%pwd

'd:\\ML\\Projects\\Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\ML\\Projects\\Text-Summarizer'

In [12]:
# defining new entity
from dataclasses import dataclass
from pathlib import Path

# decorator ensures that it's umimmutable (once created then it can't be changed/modified)
@dataclass(frozen=True)
class DataValidationConfig:
    """Data Validation configuration class."""
    root_dir: Path          # root directory where validation is performed
    STATUS_FILE: str        # path to the file where validation status will be written
    ALL_REQUIRED_FILES: list  # list of required files to check for existence



*First step is check whether the ingested data has all required files or not.*

In [13]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [14]:
# configuration manager definition

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root]) # if error -> change self.config[artifacts_root]
        
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        
        create_directories([config.root_dir])
        
        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            STATUS_FILE = config.STATUS_FILE,
            ALL_REQUIRED_FILES = config.ALL_REQUIRED_FILES,
        )
        
        return data_validation_config

In [15]:
import os
from src.textSummarizer.logging import logger

In [16]:
# components definition

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        
    # validate that all required files exist
    def validate_all_files_exist(self) -> bool:
        try:
            validation_status = None
            
            all_files = os.listdir(os.path.join("artifacts", "data_ingestion", "samsum_dataset"))
            
            for file in all_files:
                if file not in self.config.ALL_REQUIRED_FILES:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                        
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
            
            return validation_status
        
        except Exception as e:
            raise e

Further validation can also contain:  
    - the cols count check or check each feature's name.  
    - total row count check  
    - each feature's datatype check and so on.  

In [ ]:
# pipeline definition

try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_files_exist()
except Exception as e:
    raise e


[2024-12-08 17:51:09,711: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-08 17:51:09,726: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-08 17:51:09,729: INFO: common: created new directory at: artifacts]
[2024-12-08 17:51:09,731: INFO: common: created new directory at: artifacts/data_validation]
